In [1]:
import spikeinterface as si
import spikeinterface.extractors as se 
import spikeinterface.preprocessing as spre
import spikeinterface.widgets as sw

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

import warnings
warnings.simplefilter("ignore")

%matplotlib widget

In [3]:
# base_folder = Path(r"/scratch2/weka/wanglab/prevosto/data/sc012/sc012_0123/sc012_0123_001")
# file_path = r"/scratch2/weka/wanglab/prevosto/data/sc012/sc012_0123/sc012_0123_001/Record Node 101"

# DONE
# sc014\sc014_0324\sc014_0324_001
# sc014\sc014_0325\sc014_0325_001

base_folder = Path(r"D:\Vincent\Data\sc012\sc012_0120\sc012_0120_001")
file_path = base_folder.joinpath("Record Node 101")

In [4]:
recording = se.read_openephys(file_path, stream_id='1')

In [5]:
# recording_baseline = si.SelectSegmentRecording(recording, segment_indices=0)

In [6]:
channel_ids = recording.get_channel_ids()
fs = recording.get_sampling_frequency()
num_chan = recording.get_num_channels()
num_segments = recording.get_num_segments()

print(f'Channel ids: {channel_ids}')
print(f'Sampling frequency: {fs}')
print(f'Number of channels: {num_chan}')
print(f"Number of segments: {num_segments}")

Channel ids: ['AP1' 'AP2' 'AP3' 'AP4' 'AP5' 'AP6' 'AP7' 'AP8' 'AP9' 'AP10' 'AP11'
 'AP12' 'AP13' 'AP14' 'AP15' 'AP16' 'AP17' 'AP18' 'AP19' 'AP20' 'AP21'
 'AP22' 'AP23' 'AP24' 'AP25' 'AP26' 'AP27' 'AP28' 'AP29' 'AP30' 'AP31'
 'AP32' 'AP33' 'AP34' 'AP35' 'AP36' 'AP37' 'AP38' 'AP39' 'AP40' 'AP41'
 'AP42' 'AP43' 'AP44' 'AP45' 'AP46' 'AP47' 'AP48' 'AP49' 'AP50' 'AP51'
 'AP52' 'AP53' 'AP54' 'AP55' 'AP56' 'AP57' 'AP58' 'AP59' 'AP60' 'AP61'
 'AP62' 'AP63' 'AP64' 'AP65' 'AP66' 'AP67' 'AP68' 'AP69' 'AP70' 'AP71'
 'AP72' 'AP73' 'AP74' 'AP75' 'AP76' 'AP77' 'AP78' 'AP79' 'AP80' 'AP81'
 'AP82' 'AP83' 'AP84' 'AP85' 'AP86' 'AP87' 'AP88' 'AP89' 'AP90' 'AP91'
 'AP92' 'AP93' 'AP94' 'AP95' 'AP96' 'AP97' 'AP98' 'AP99' 'AP100' 'AP101'
 'AP102' 'AP103' 'AP104' 'AP105' 'AP106' 'AP107' 'AP108' 'AP109' 'AP110'
 'AP111' 'AP112' 'AP113' 'AP114' 'AP115' 'AP116' 'AP117' 'AP118' 'AP119'
 'AP120' 'AP121' 'AP122' 'AP123' 'AP124' 'AP125' 'AP126' 'AP127' 'AP128'
 'AP129' 'AP130' 'AP131' 'AP132' 'AP133' 'AP134' 'AP135' '

In [8]:
probe = recording.get_probe()
print(probe)

Probe - 384ch - 1shanks


In [9]:
print(type(probe))

<class 'probeinterface.probe.Probe'>


In [ ]:
sw.plot_probe_map(recording)

In [13]:
print("Properties:\n", list(recording.get_property_keys()))

Properties:
 ['gain_to_uV', 'offset_to_uV', 'channel_name', 'contact_vector', 'location', 'group', 'inter_sample_shift']


In [14]:
print(recording._properties.keys())

dict_keys(['gain_to_uV', 'offset_to_uV', 'channel_name', 'contact_vector', 'location', 'group', 'inter_sample_shift'])


### Phase shift

In [7]:
recording = spre.phase_shift(recording)

### High-pass filter

In [8]:
recording_f = spre.highpass_filter(recording, freq_min=300)
# recording = spre.bandpass_filter(recording, freq_min=300, freq_max=6000)

In [ ]:
recording.annotate(is_filtered=False)
recording_f.annotate(is_filtered=True)

### Common Median Reference

In [9]:
recording_baseline = si.SelectSegmentRecording(recording_f, segment_indices=0)
# recording_baseline = si.concatenate_recordings([recording_baseline])
recording_baseline

SelectSegmentRecording: 384 channels - 1 segments - 30.0kHz - 659.916s

In [10]:
recording_b_cmr = spre.common_reference(recording_baseline, reference='global', operator='median')

In [ ]:
w = sw.plot_timeseries({"raw": recording, "filt": recording_f, "common": recording_b_cmr}, segment_index=0,
                        clim=(-50, 50), time_range=[10, 10.1], order_channel_by_depth=True,
                        backend="ipywidgets")

In [11]:
job_kwargs = dict(n_jobs=30, chunk_duration="1s", progress_bar=True)

In [12]:
# if (base_folder / "preprocessed").is_dir():
#     recording_saved = si.load_extractor(base_folder / "preprocessed")
# else:
# recording_saved = recording_cmr.save(folder=base_folder / "preprocessed", **job_kwargs)
    
recording_saved = recording_b_cmr.save(folder=base_folder / "preprocessed", format='zarr', **job_kwargs)

Using default zarr compressor: Blosc(cname='zstd', clevel=5, shuffle=BITSHUFFLE, blocksize=0). To use a different compressor, use the 'compressor' argument
write_zarr_recording with n_jobs = 30 and chunk_size = 30000


write_zarr_recording:   0%|          | 0/660 [00:00<?, ?it/s]

If we inspect the `preprocessed` folder, we find that a few files have been saved. Let's take a look at what they are:

In [13]:
recording_saved

ZarrRecordingExtractor: 384 channels - 1 segments - 30.0kHz - 659.916s

In [ ]:
# !ls {base_folder}\preprocessed

In [ ]:
# print(f'Cached channels ids: {recording_saved.get_channel_ids()}')
# print(f'Channel groups after caching: {recording_saved.get_channel_groups()}')

In [ ]:
# recording_loaded = si.load_extractor(base_folder / "preprocessed")

In [ ]:
# print(f'Loaded channels ids: {recording_loaded.get_channel_ids()}')
# print(f'Channel groups after loading: {recording_loaded.get_channel_groups()}')

In [ ]:
# w = sw.plot_timeseries({"preprocessed": recording_cmr, "saved": recording_saved, "loaded": recording_loaded},
#                         clim=(-50, 50), mode="line",
#                         backend="ipywidgets")